In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [2]:
from keras.models import load_model

2022-12-17 12:44:09.368542: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import pandas as pd

### Get weights from pre-trained Keras model

In [4]:
keras_DNN = load_model("CF_Mortality.model")

2022-12-17 12:45:07.553734: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
keras_DNN.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 128)]             0         
                                                                 
 dropout_21 (Dropout)        (None, 128)               0         
                                                                 
 dense_25 (Dense)            (None, 128)               16512     
                                                                 
 dropout_22 (Dropout)        (None, 128)               0         
                                                                 
 dense_26 (Dense)            (None, 64)                8256      
                                                                 
 dropout_23 (Dropout)        (None, 64)                0         
                                                                 
 dense_27 (Dense)            (None, 64)                4160

In [6]:
keras_DNN_weights = keras_DNN.get_weights()

### Reconstruct Keras model in PyTorch


In [7]:
'''
def DNN_Simple(shape):
    X_in = Input(shape=(shape,))
    H2 = Dense(128, activation='relu', kernel_regularizer=l2(5e-4))(X_in)
    H3 = Dropout(0.5)(H2)
    H4 = Dense(64, activation='relu', kernel_regularizer=l2(5e-4))(H3)
    H5 = Dropout(0.2)(H4)
    H6 = Dense(64, activation='relu', kernel_regularizer=l2(5e-4))(H5)
    H7 = Dropout(0.2)(H6)
    H8 = Dense(48, activation='relu', kernel_regularizer=l2(5e-4))(H7)
    H9 = Dropout(0.2)(H8)
    H10 = Dense(16, activation='relu', kernel_regularizer=l2(5e-4))(H9)
    H11 = Dropout(0.5)(H10)
    Y = Dense(3, activation='softmax')(H11)
    model = Model(inputs=X_in, outputs=Y)
    model.compile(optimizer=optimizers.Adam(decay=0.01),loss='categorical_crossentropy'
                  ,metrics=['accuracy']) 
    return model
'''

"\ndef DNN_Simple(shape):\n    X_in = Input(shape=(shape,))\n    H2 = Dense(128, activation='relu', kernel_regularizer=l2(5e-4))(X_in)\n    H3 = Dropout(0.5)(H2)\n    H4 = Dense(64, activation='relu', kernel_regularizer=l2(5e-4))(H3)\n    H5 = Dropout(0.2)(H4)\n    H6 = Dense(64, activation='relu', kernel_regularizer=l2(5e-4))(H5)\n    H7 = Dropout(0.2)(H6)\n    H8 = Dense(48, activation='relu', kernel_regularizer=l2(5e-4))(H7)\n    H9 = Dropout(0.2)(H8)\n    H10 = Dense(16, activation='relu', kernel_regularizer=l2(5e-4))(H9)\n    H11 = Dropout(0.5)(H10)\n    Y = Dense(3, activation='softmax')(H11)\n    model = Model(inputs=X_in, outputs=Y)\n    model.compile(optimizer=optimizers.Adam(decay=0.01),loss='categorical_crossentropy'\n                  ,metrics=['accuracy']) \n    return model\n"

In [7]:
class DNN(nn.Module):
    def __init__(self, input_shape):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(input_shape, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 48)
        self.fc5 = nn.Linear(48, 16)
        self.output = nn.Linear(16, 2)
        
        self.dropout_50 = nn.Dropout(0.5)
        self.dropout_20 = nn.Dropout(0.2)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout_50(x)
        x = F.relu(self.fc2(x))
        x = self.dropout_20(x)
        x = F.relu(self.fc3(x))
        x = self.dropout_20(x)
        x = F.relu(self.fc4(x))
        x = self.dropout_20(x)
        x = F.relu(self.fc5(x))
        x = self.dropout_50(x)
        return F.softmax(self.output(x))

### Transfer pre-trained weights from Keras model to PyTorch

In [8]:
pytorch_DNN = DNN(128)

In [9]:
pytorch_DNN

DNN(
  (fc1): Linear(in_features=128, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=48, bias=True)
  (fc5): Linear(in_features=48, out_features=16, bias=True)
  (output): Linear(in_features=16, out_features=2, bias=True)
  (dropout_50): Dropout(p=0.5, inplace=False)
  (dropout_20): Dropout(p=0.2, inplace=False)
)

In [10]:
pytorch_DNN.fc1.weight.data = torch.from_numpy(np.transpose(keras_DNN_weights[0]))
pytorch_DNN.fc1.bias.data = torch.from_numpy(keras_DNN_weights[1])
pytorch_DNN.fc2.weight.data = torch.from_numpy(np.transpose(keras_DNN_weights[2]))
pytorch_DNN.fc2.bias.data = torch.from_numpy(keras_DNN_weights[3])
pytorch_DNN.fc3.weight.data = torch.from_numpy(np.transpose(keras_DNN_weights[4]))
pytorch_DNN.fc3.bias.data = torch.from_numpy(keras_DNN_weights[5])
pytorch_DNN.fc4.weight.data = torch.from_numpy(np.transpose(keras_DNN_weights[6]))
pytorch_DNN.fc4.bias.data = torch.from_numpy(keras_DNN_weights[7])
pytorch_DNN.fc5.weight.data = torch.from_numpy(np.transpose(keras_DNN_weights[8]))
pytorch_DNN.fc5.bias.data = torch.from_numpy(keras_DNN_weights[9])
pytorch_DNN.output.weight.data = torch.from_numpy(np.transpose(keras_DNN_weights[10]))
pytorch_DNN.output.bias.data = torch.from_numpy(keras_DNN_weights[11])

### Test if Keras and PyTorch models behave similarly

In [11]:
pytorch_DNN.eval()

DNN(
  (fc1): Linear(in_features=128, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=48, bias=True)
  (fc5): Linear(in_features=48, out_features=16, bias=True)
  (output): Linear(in_features=16, out_features=2, bias=True)
  (dropout_50): Dropout(p=0.5, inplace=False)
  (dropout_20): Dropout(p=0.2, inplace=False)
)

In [12]:
test_input = torch.rand(128)

In [13]:
test_input

tensor([0.3444, 0.9813, 0.7785, 0.0401, 0.3178, 0.0788, 0.0755, 0.8693, 0.1432,
        0.9060, 0.9657, 0.1413, 0.5917, 0.5230, 0.9480, 0.5470, 0.3413, 0.9207,
        0.7817, 0.7178, 0.7958, 0.9180, 0.0037, 0.2990, 0.2454, 0.6125, 0.5545,
        0.6472, 0.8791, 0.5012, 0.0671, 0.7632, 0.8862, 0.7937, 0.9149, 0.6799,
        0.7679, 0.5965, 0.8130, 0.5568, 0.7320, 0.6051, 0.9923, 0.7988, 0.0129,
        0.7642, 0.3316, 0.1957, 0.9704, 0.7133, 0.5589, 0.5751, 0.9515, 0.5327,
        0.7401, 0.9360, 0.2170, 0.9183, 0.1435, 0.7323, 0.1798, 0.6054, 0.9977,
        0.6405, 0.0051, 0.4881, 0.2498, 0.9334, 0.1217, 0.1179, 0.0584, 0.2180,
        0.6156, 0.2785, 0.0810, 0.8296, 0.3665, 0.9396, 0.4603, 0.6959, 0.7657,
        0.6067, 0.1079, 0.4412, 0.6775, 0.2643, 0.1774, 0.1865, 0.8165, 0.1248,
        0.0606, 0.8652, 0.2587, 0.8120, 0.3374, 0.8718, 0.9300, 0.2189, 0.7270,
        0.9533, 0.1167, 0.5172, 0.9277, 0.6798, 0.9896, 0.1638, 0.5077, 0.9047,
        0.6243, 0.6280, 0.8183, 0.3287, 

In [14]:
pytorch_DNN(test_input.float())

/var/folders/cy/nvtp62xj7_188hvgxm3r35hm0000gn/T/ipykernel_8348/3945986980.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(self.output(x))


tensor([0.9708, 0.0292], grad_fn=<SoftmaxBackward0>)

In [15]:
test_input_np = test_input.numpy()

In [16]:
test_input_np

array([0.34444815, 0.98134845, 0.7784901 , 0.04014856, 0.3177709 ,
       0.0787943 , 0.0755015 , 0.86933136, 0.14321393, 0.906044  ,
       0.96566004, 0.14130646, 0.5916648 , 0.52301526, 0.94802105,
       0.54704887, 0.3413403 , 0.9207316 , 0.7817209 , 0.71782655,
       0.795812  , 0.91804063, 0.00367725, 0.2990077 , 0.24542099,
       0.612544  , 0.5544843 , 0.6472434 , 0.87913364, 0.50120884,
       0.06712216, 0.7632213 , 0.88622284, 0.7936504 , 0.9148641 ,
       0.6799495 , 0.76788527, 0.5964951 , 0.81298876, 0.55682194,
       0.7319585 , 0.60513514, 0.9923075 , 0.79879034, 0.01287615,
       0.7641729 , 0.33162028, 0.19571972, 0.9703503 , 0.7132627 ,
       0.5589338 , 0.57507455, 0.95145875, 0.5327248 , 0.74010134,
       0.93602276, 0.21703333, 0.91828215, 0.14354217, 0.7323243 ,
       0.179784  , 0.605447  , 0.99767494, 0.6404511 , 0.00507313,
       0.48806643, 0.24979359, 0.93336   , 0.12174898, 0.11792588,
       0.05844009, 0.21797788, 0.61563385, 0.27853143, 0.08098

In [17]:
keras_DNN.predict(test_input_np.reshape(1, -1))

1/1 [==============================] - 0s 131ms/step


array([[0.97076607, 0.02923388]], dtype=float32)

### Load data

In [1]:
data_df = pd.read_csv("matt_metadata_norm.csv", sep='\t')

NameError: name 'pd' is not defined